In [1]:
%pip install fastapi pydantic toml

Note: you may need to restart the kernel to use updated packages.


In [2]:
# from rs import skill_check

# def test_skill_check(metadata):
#     print(skill_check("sanity", None, metadata))

# metadata = {
#     "character": {"sanity":30},
#     "symptom":[
#         "incoherent speech",
#         "uncontrollable twitching, trembling",
#         "delusions of persecution",
#         "strange appetites (dirt, clay, etc...)",
#         "scratches, punches, and bruises all over the body"
#     ]
# }

# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)

# print(metadata["character"])


In [ ]:
import re
import tomllib
import functools
from typing import Literal

import ipywidgets as widgets
from IPython.display import display

from dotenv import load_dotenv

load_dotenv()

from rs import Message, Config, chat, skill_check
import d20

action_pattern = re.compile(r"(\d+). *(.+)")
skill_pattern = re.compile(r"[\w_]+")
skill_difficulty_pattern = re.compile(r"(easy|normal|hard)")

def next_round(b):
    global current_round, initial_messages
    current_round += 1
    initial_messages = config.initial_messages(current_round, config.metadata)

    print(initial_messages[-1].content)
    display(get_controls(initial_messages[-1]))

def get_user_message_tail():
    count = 0
    for msg in initial_messages:
        if msg.role == "user":
            count += 1

    is_final =  ", FINAL ROUND" if  current_round == len(config.rounds) else ""
    
    # print(config.metadata["character"])
    
    if "tags" in config.metadata["character"] and "insane" in config.metadata["character"]["tags"]:
        return f"TURN: {count + 1} {is_final} (I'm insane!)"
    
    return f"TURN: {count + 1} {is_final}"
    

def do_chat(role:Literal["user", "system", "assistant"], content: str):
    print(f"{content}\n\n")

    initial_messages.append(Message(role=role, content=content))
    msg = chat(initial_messages, lambda delta: print(delta, end=""))

    initial_messages.append(msg)
    display(get_controls(msg))


def on_skill(b, skill:str, difficulty:str):
    # print(config.metadata["character"])
    content = skill_check(skill, difficulty, config.metadata)
    content += f"\n{get_user_message_tail()}"
    do_chat("user", content)

def on_action(b, index:int):
    content = f"I select option {index}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_input(b, input:widgets.Text):
    content = f"{input.value}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_action(b):
    hbox = widgets.HBox(layout=widgets.Layout(width="100%"))
    action_text = widgets.Text(layout=widgets.Layout(width="100%"))
    submit = widgets.Button(description="SUBMIT", layout=widgets.Layout(width="fit-content"))
    submit.on_click(functools.partial(on_custom_input, input=action_text))
    hbox.children += (action_text, submit) 
    display(hbox)

def get_controls(msg:Message) -> widgets.Box:
    dict = msg.dict()
    buttons = []
    if "possible actions" in dict.keys():
        matches = action_pattern.finditer(dict["possible actions"])
        for match in matches:
            btn = widgets.Button(description=match.group(1), tooltip=match.group(2), layout=widgets.Layout(width="auto"))
            btn.on_click(functools.partial(on_action, index=int(match.group(1))))
            buttons.append(btn)
    elif "skill" in dict.keys() and "difficulty" in dict.keys():
        match = skill_pattern.match(dict["skill"])
        skill = match.group(0)
        match = skill_difficulty_pattern.match(dict["difficulty"])
        difficulty = match.group(0)
        skill_button = widgets.Button(description=f"{skill.upper()} [{difficulty.upper()}]", layout=widgets.Layout(width="auto"))
        skill_button.on_click(functools.partial(on_skill, skill=skill, difficulty=difficulty))
        buttons.append(skill_button)
    elif "summary" in dict.keys():
        # add summary as a memory to the metadata
        # it will be used in the next round.
        config.metadata["memories"] = []
        config.metadata["memories"].append(dict["summary"])
        next_btn = widgets.Button(description="NEXT ROUND", layout=widgets.Layout(width="auto"))
        next_btn.on_click(next_round)
        buttons.append(next_btn)
    
    custom_action_button = widgets.Button(description="CUSTOM")
    custom_action_button.on_click(on_custom_action)
    buttons.append(custom_action_button)

    return widgets.HBox(buttons)

# open config file
config_name = "data/coc_lot_3636.toml"
config: Config | None = None
with open(config_name, "rb") as f:
    data = tomllib.load(f)
    config = Config(**data)

current_round = 0
initial_messages = config.initial_messages(current_round, config.metadata)

print(initial_messages[-1].content)
display(get_controls(initial_messages[-1]))

You arrived at an old self-storage facility by invitation from the manager, who has an storage room up for auction due to the death of its owner. It is not your first rodeo with these forms of auctions, like a box of chocolate, no one knows of the potential treasure that hides behind the cold roller shutter. As you and other interested buyers gathered around the auctioneer, a worker carrying a bolt cutter snapped off the rusty lock, unveiling a storage room cluttered with various old furniture and antiquities. Without any opportunity for the bidders to take a clear look, the auctioneer began the bid at $50. 

What is your bid?

possible actions:
1. $100.
2. $150.
3. $200.


I select option 2. TURN: 1 


You confidently raise your hand and bid $150, catching the attention of a few curious onlookers. The auctioneer acknowledges your bid with a nod and immediately raises his voice, "We have $150 from the gentleman over here! Do I hear $200?"

After a brief moment, a woman in the back, dressed in a long, dark coat, raises her hand, "Two hundred!"

The auctioneer points towards her, "We're now at $200! Can I get $250?"

What's your next move?

possible actions:
1. Bid $250.
2. Bid $300.
3. Hold your bid and wait.

I select option 1. TURN: 2 


With determination, you raise your hand again, "Two fifty!" The auctioneer's eyes light up as he quickly acknowledges your bid, "We have $250 from the gentleman in the front! Do I hear $300?"

The woman in the back hesitates for a moment but then raises her hand, "Three hundred!" It seems she is quite interested in the storage as well.

The auctioneer looks back at you, "We're at $300! Will you go to $350?"

What's your next move?

possible actions:
1. Bid $350.
2. Bid $400.
3. Hold your bid and wait.

I select option 1. TURN: 3 


"Three fifty!" you call out, not willing to back down easily. The auctioneer nods in your direction, "Three hundred and fifty dollars from the gentleman here! Do I hear $400?"

The woman in the back pauses, giving you a long, calculating look before finally shaking her head and stepping back, no longer participating in the bidding.

The auctioneer waits for a few more seconds to see if there are any more bids. Seeing no further interest, he starts his final call, "Going once, going twice... and sold to the gentleman for $350!"

Congratulations, you've won the bid for the storage room. As the crowd disperses, the auctioneer approaches you to finalize the paperwork and payment.

With the keys in your hand, you stand before the storage room, its contents now yours to explore. The door creaks open, revealing shadows cast by the late afternoon sun, hinting at forgotten stories and hidden treasures within.

summary: Kyle Jackson successfully bids $350 for a stor


Memories:
"""
- Kyle Jackson successfully bids $350 for a storage room at an auction, outbidding others, including a mysterious woman. The contents of the storage room now await his exploration.
"""


You follow the manager through the myriad hallways to his dimly lit office, where the furnishings and appliances remain unchanged since the 90’s. He often reminds you that the facility is in fact even older, and has been here since the 60’s. You made a quick phone call to Alex, a man you owe money to, and explained to him that you will pay him back in a few days once you have it, to which he responded with distasteful threats. The manager took out an old tin box and placed it on his desk and reached towards you with an open palm, gesturing for you to pay up, which you obliged. Upon handing you a key for your newly owned storage room, he urged you to watch some old security footage.

Through his CRT screen, he explains to you that the previous owner visited the storage room every day for 

...

I select option 1. TURN: 2 


You decide to continue following Professor Dagger in silence, focusing intently on your surroundings. The narrow corridor seems to close in around you, the darkness almost palpable, as you tread carefully on the uneven ground. Your eyes dart around, trying to catch any detail that might reveal more about this hidden passage beneath the storage unit.

The silence between you and the Professor is heavy, filled with anticipation and a hint of fear. The only sounds are your footsteps and the occasional drip of water from the ceiling, creating an eerie rhythm that accompanies your journey into the unknown.

As you proceed, the corridor finally begins to widen, leading into a larger, open space. The air here is thick with the smell of decay, confirming the Professor's earlier statement. The beam from his flashlight reveals a shocking sight: the near-withered body of a woman, lying in the center of the room over a pentagram made of a strange red fluid. Her face i

I am making a SANITY check against my own sanity points: 8.
And... FAILED! I lost 3 sanity points.
TURN: 3  (I'm insane!)


As you gaze upon the grotesque sight before you, something inside you snaps. The sheer horror of the withered body, the sinister pentagram beneath it, and the unnatural absence of a face overwhelms your senses. Your heart races, your vision blurs, and a cold sweat breaks out across your forehead. You struggle to maintain your grip on reality, but the terror of what lies before you is too much to bear.

You stagger back, gasping for air, your mind reeling from the shock. The world seems to tilt and spin around you, and for a moment, you fear you might succumb to the darkness that threatens to engulf your consciousness.

Professor Dagger, noticing your distress, rushes to your side, his voice a distant echo in your mind. "Kyle! Focus on my voice. You must not let it consume you. Fight it!" His words barely register as you fight to regain control, the horror of the s

I select option 1. TURN: 4  (I'm insane!)


With every ounce of courage you can muster, you decide to approach the podium cautiously, doing your best to avert your eyes from the withered body lying in the center of the room. The strange red fluid forming the pentagram beneath it seems to shimmer in the dim light, adding an otherworldly quality to the already eerie scene.

As you move closer to the podium, your focus narrows to the object of your quest—the fourth book. It sits atop the podium, its cover worn by time, yet it exudes a palpable sense of power and forbidden knowledge. You can't help but feel that this book holds the key to changing your fortunes, but at what cost?

Reaching out with a trembling hand, you grasp the book, the leather cool and slightly sticky to the touch. The moment the book leaves the podium, a sudden change occurs. The strange red fluid forming the pentagram begins to evaporate, disappearing into thin air as if it was never there.

This action, however, has

I select option 1. TURN: 5  (I'm insane!)


Fueled by primal fear and the instinct to survive, you turn on your heels and sprint towards the corridor, the horrifying sounds of the creature's tentacles slithering across the floor echoing behind you. Your heart pounds in your chest, your breaths come in ragged gasps, and the dimly lit corridor seems to stretch into infinity. The terror of being chased by something so utterly alien and malevolent propels you forward at breakneck speed, every ounce of your being focused on escape.

As you run, the memories of your training as a marine surge to the forefront of your mind, lending you the mental fortitude to push your body beyond its limits. Your footsteps echo loudly in the narrow passage, a stark contrast to the sinister silence that seems to have fallen behind you. Dare you hope that you've outpaced the creature?

Finally, you see the entrance to the storage unit ahead, the faint light from the outside world offering a glimmer of hope. Wi

I select option 3. TURN: 6  (I'm insane!)


Desperation takes hold as you decide to call out for help, your voice echoing through the storage unit and possibly beyond its confines. "Help! Is anyone there? Please, I need help!" you shout, your voice cracking with the strain of your recent terror and exertion.

You pause, listening intently for any sign of response, any indication that someone might have heard your pleas. The silence that follows feels oppressive, a stark reminder of your isolation in this nightmarish scenario. Just as you're about to give up hope, you hear the faint sound of footsteps approaching from outside the storage unit.

The footsteps grow louder, more deliberate, until they stop just outside the door. You hold your breath, not sure whether to be relieved or terrified. Then, you hear a key turning in the lock, and the door swings open to reveal a middle-aged woman standing in the doorway. Her expression is one of concern mixed with confusion as she takes in your 